In [ ]:
import os.path
from os import path
import requests
import tarfile
import shutil

In [ ]:
PATH_TO_RESOURCES = "../resources"

## Récupération des ressources

**param force** : 1 si l'on veut télécharger la donnée m

1. Controlle l'existence d'un dossier de **resources** situé à la racine du projet
2. Si ce dossier existe et que **force == 0** alors il n'y a rien à faire
3. Sinon, le dossier est crée, l'archive est téléchargée et décompréssée puis supprimée.

In [ ]:
# Check if a local resources file exists.
# If not or force=True, creates the files and uploads data.
def init_resources(force = False):
    PATH_TO_RESOURCES = "../resources"
    
    if (path.exists(PATH_TO_RESOURCES) and path.isdir(PATH_TO_RESOURCES)):
        if not force:
            print("Folder 'resources' found.")
            return
        else:
            shutil.rmtree(PATH_TO_RESOURCES)
    
    # Create folder
    os.mkdir(PATH_TO_RESOURCES)
    print("[1/4] Folder '../resources' created.")
    
    # Download data
    url_data = 'https://www.lrde.epita.fr/~jchazalo/SHARE/dobble-symbols-dataset-train.tar.gz'
    url_dobble_img = 'https://boardgamereview.co.uk/wp-content/uploads/2020/02/Dobble-cards-pile-1.png'
    TAR_PATH = os.path.join(PATH_TO_RESOURCES, 'dobble-symbols-dataset-train.tar.gz')
    IMG_PATH = os.path.join(PATH_TO_RESOURCES, 'dobble_ref.png')
    response = requests.get(url_data, stream=True)
    if response.status_code == 200:
        with open(TAR_PATH, 'wb') as f:
            f.write(response.raw.read())
    response = requests.get(url_dobble_img, stream=True)
    if response.status_code == 200:
        with open(IMG_PATH, 'wb') as f:
            f.write(response.raw.read())
            print("[2/4] Resources files uploaded.")
    
    # Extract from tar
    tar = tarfile.open(TAR_PATH)
    tar.extractall(PATH_TO_RESOURCES) # specify which folder to extract to
    tar.close()
    print("[3/4] Resources files extracted.")
    
    # Delete dowloaded tar
    os.remove(TAR_PATH)
    print("[4/4] Tarball removed.")

In [ ]:
init_resources(0)

## Transformateurs

### Extraction de la zone d'intérêt
1. Récupération des données d'entrainement
2. Calcul des masques
3. Extraction des pixels à utiliser dans chaque image

#### Explications

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [ ]:
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

In [ ]:
# define a function to compute the mask of a bubble
def symbol2mask(symbol):
    return np.sum(symbol, axis=2) < 255 * 3

In [ ]:
symbol_files = !ls $PATH_TO_RESOURCES/**/c*.png  | sort
symbol_files

In [ ]:
symbols = [load_image(symbol_file) for symbol_file in symbol_files]

In [ ]:
symbols_gray = [ cv2.imread(symbol_file, cv2.IMREAD_GRAYSCALE) for symbol_file in symbol_files]

In [ ]:
symbols_masks = [symbol2mask(symbol) for symbol in symbols]

## Moments

In [ ]:
import cv2
from math import copysign, log10

In [ ]:
# Calculate Moments
moments = cv2.moments(symbols_gray[0])
# Calculate Hu Moments
huMoments = cv2.HuMoments(moments)

# Log scale hu moments
for i in range(0,7):
    huMoments[i] = -1* copysign(1.0, huMoments[i]) * log10(abs(huMoments[i]))

huMoments

In [ ]:
huMoments

In [ ]:
X_gray = [cv2.cvtColor(x, cv2.COLOR_BGR2GRAY) for x in symbols]